# Лабораторная работа "Фемптосекундный  лазер"

## В  настоящем блокноте представлена  программа для  выполнения следующих задач:<br>
### 1)Определяете период следования и частоту повторений импульсов лазера  
### 2)Определяете время наростания фронта импульсов от 10% до 90% на обоих фотоприёмниках
### 3)Привести спектр и описать его, определить ширину спектра 
### 4)Привести среднюю мощность на выходе из лазера и кристалла, определить эффективность преобразования во 2 гармонику 
### 5)Проверить, является ли импульс спектрально ограниченным
### 6)Оценить энергию в одном импульсе
### 7)Посчитать пиковую мощность 
### 8)Посчитать пиковую интенсивность при фокусировке импульса в пятно диаметром 8 мкм

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.signal import argrelextrema
from scipy.signal import find_peaks, peak_widths,savgol_filter
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit
import copy
import math
from scipy.stats import t